In [1]:
# Copyright (c) 2012-2023, NECOTIS
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
#
#  - Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
#  - Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#  - Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
# IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT
# NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA,
# OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

# Authors: Emmanuel Calvet, Bertrand Reulet,  Jean Rouat (advisors)
# Date: December 1th, 2023
# Organization: Groupe de recherche en Neurosciences Computationnelles et Traitement Intelligent des Signaux (NECOTIS),
# Université de Sherbrooke, Canada


In [2]:
# Import library and models
# NB: you need to import binary_model_design into the python path
import sys
import os
import importlib

# Get the module path and add it to python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Setting the module 
model_path = os.path.join(module_path, 'model')
MODULE_PATH = f"{model_path}/__init__.py"
MODULE_NAME = "binary_model" # /!\ if you change this, you must also modify the call of the module in all dependant files!
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)
    
# Standard library
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# The model
from tasks.whitenoise import whiteNoiseGauss
from binary_model.binaryModel import binaryModel
from binary_model.utils import (initRandomArchitecture, 
                            initGaussianWeights, dataPath,
                            dataPathPerf, my_corrcoef, rescale)

In [3]:
# The experiment
sim = 'performance_task'
experiment = 'memory_white-noise'

N = 10000 # Reservoir size. Values tested in the paper: [100, 1000, 10000]
duration = 2000
nb_reservoir = 20 # Number of reservoir par value of sigma
nb_trial = 5 # Number of different state initialization of the same reservoir
K = 16 # Connectivity degree of the graph. [1, 2, 3, 4, 5, 6, 8, 16]
meanWeight = -0.1 # Average weights for the normal distribution: [-0.1, 0.1]

# List of standard deviation sigma (of the normal distribution)
sigmas = [
            0.01, 0.01247, 0.01555, 0.01939, 0.02418, 0.03016, 0.03761, 0.0469, 0.05848,
            0.07293, 0.09094, 0.11341, 0.14142, 0.17636, 0.21992, 0.27425, 0.342, 0.42648,
            0.53183, 0.66321, 0.82704, 1.03134, 1.28611, 1.60381, 2.0, 2.77899, 3.8614,
            5.36539, 7.45519, 10.0, 10.35895, 12.9155, 14.39371, 16.68101, 20.0, 21.54435,
            27.82559, 35.93814, 46.41589, 59.94843, 77.42637, 100.0
        ]

# Values for testing purpose
nb_sigmas = 2
exponents = np.linspace(-2, 2, nb_sigmas)
sigmas = np.power(10, exponents)

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

The model will be evaluated on 2 values of sigma(W):
sigma(W) = [1.e-02 1.e+02]


In [4]:
mu = meanWeight
# Parameters of the task --------------------------------------
delays = [-10, -2] # Parameter controlling difficulty. Value tested in the paper: [-18, -14, -10, -6, -2]

# Parameter of the training -----------------------------
nbSample = 4000 # Number sample for the white-noise time serie
tTrain = nbSample #  Time of training
nbtrial = 5 # Number of trial for the training and tasks
tDiscard = 500 # Discard 500 time step of transient
tConv = 0 
optionData = {'spikeCount': False, 'indexActive': False} 
kwargs = {'optionRun':optionData} # kwargs for the experiment

# Parameters of the network -----------------------------
inSize = 1 # Number of input neurons
outSize = 1 # Number of output neurons
I = 0.5 # Proportion of neuron of the reservoir to which the input/output is connected
output_option =  'exclusion' # Option to obtain a non-overlapping set of neurons connected to input and readout
kwargsOut = {'option':output_option,
            'I':I}

# Options of the readout --------------------------------
learning = 'online'
epoch = 4000
optimizer = 'pytorch' 
activation = 'sigmoid'
optim = 'adam'
loss = 'MSE'
scheduler = False
alpha = 0.001

if 'ridge' in optimizer:
    kwargs.update({'beta':1E-08})
elif optimizer == 'pseudoinverse':
    kwargs.update({'activation':activation})
kwargs.update({'optionCost':'mse'})
if optimizer == 'ANN':
    kwargs.update({'alpha':1E-04,
                    'nbiter':epoch,
                    'activation':activation})
elif optimizer == 'pytorch':
    kwargs.update({'optionOptim':{'alpha':alpha,
                        'epoch':epoch,
                        'activation':activation,
                        'optim':optim,
                        'loss':loss,
                        'scheduler':scheduler,
                        }
            })

tag = f"{experiment}_{optimizer}_{activation}_output-{output_option}"

# Dictionary of result ---------------------------------
path = dataPathPerf(sim, experiment, optimizer=optimizer, activation=activation, nbtrial=nb_trial)
name = path+f'\dicError_{tag}_nbReservoir{nb_reservoir}_N{N}_K{K}_W{mu}_T{nb_trial}'
try:
    dicResult = np.load(name+'.npy', allow_pickle='TRUE').item()
    corr_delay = dicResult['corr']
    infoReservoir = dicResult['infoReservoir']
    print('File loaded:', name)
except:
    print('File not loaded, will create one:', name)
    corr_delay = {delay: {} for delay in delays}
    infoReservoir = {delay: {} for delay in delays}
    dicResult = {'corr':corr_delay,
                    'delay':delays,
                    'nbtrial':nb_trial,
                    'infoReservoir':infoReservoir,
                    'inputs':'all',
                    'optim': kwargs}

SCRIPT: C:\Users\Manu\Documents\GitHub\binary_model_design
File not loaded, will create one: C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/memory_white-noise/pytorch_sigmoid/nbtrial5\dicError_memory_white-noise_pytorch_sigmoid_output-exclusion_nbReservoir20_N10000_K16_W-0.1_T5


In [5]:
# Create the model
optionData = {'spikeCount':True, 'indexActive':False} # Option for saving data
brain = binaryModel(N, sim=sim, experiment=experiment)
seedConnectivity = 747
initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture
print(kwargs)

{'optionRun': {'spikeCount': False, 'indexActive': False}, 'optionCost': 'mse', 'optionOptim': {'alpha': 0.001, 'epoch': 4000, 'activation': 'sigmoid', 'optim': 'adam', 'loss': 'MSE', 'scheduler': False}}


In [ ]:
## Run the experiment
import glob

for sigma in tqdm(sigmas):
    sigma = np.round(sigma, 5)

    for res_idx in range(nb_reservoir):
        seed=res_idx*100
        print(f'Reservoir #{res_idx}, sigma={sigma}')

        # Initialize weights
        initGaussianWeights(brain, mu, sigma, seed)

        # Input neurons
        optionIn = 'global'
        kwargsIn = {}

        # Input layer initialization
        brain.addInputLayer(inSize=inSize, tag=tag)

        # Readout initialization
        brain.addReadoutLayer(outSize)
        pathBrain = dataPathPerf(sim, experiment, optimizer=optimizer, 
                                 activation=activation, nbtrial=nb_trial)+f'/sigma{sigma}/seed{seed}'

        # Check network has not been run already
        dataFile = glob.glob(pathBrain+"/metadata_"+tag+f'*N{N}_K{K}*_W{meanWeight}_*.npy')
        if len(dataFile) != 0:
            print('Network already run, next...\n \n')
            continue

        # Select delay for the targets signals
        for i, delay in enumerate(delays):
            
            # Initialize dictionary result 
            if corr_delay.get(delay, None) is None:
                corr_delay.update({delay:{}})
                infoReservoir.update({delay:{}})
            if corr_delay[delay].get(sigma, None) is None:
                corr_delay[delay].update({sigma:[]})
                infoReservoir[delay].update({sigma:[]})

            # Check if this task is already performed
            print(f'delay={delay}, sigma={sigma}')
            nb_reservoir_run = len(infoReservoir[delay][sigma])
            print(nb_reservoir_run, 'already runned circuits')
            if nb_reservoir_run >= nb_reservoir:
                print(f'Networks for delay={delay}, sigma={sigma} already run, next...\n \n')
                continue
            elif (nb_reservoir_run < nb_reservoir) and (res_idx < nb_reservoir_run):
                print(f'Network {res_idx} for delay={delay}, sigma={sigma} already run, next...\n \n')
                continue
            print('-----------------------------')
            
            # Initialize inputs and targets for training
            datas = rescale(whiteNoiseGauss(2*nbSample))
            y_train = datas[tDiscard+delay:nbSample+delay]
            x_train = datas[:tTrain] # Input of the network
            t_train = np.arange(0, tTrain)
            inputs_train = [x_train, t_train]

            corrTrials = [] # List to save performance (evulated by the correlation)
            # Compute all trials
            for k in range(nbtrial):
                # Display information
                print(f'Reservoir #{res_idx}, trial #{k}')
                print('********************')
                brain.displayInfoNetwork()
                print('********************')    
                print('White-noise, delay =', delay)

                # Input layer initialization - multiple init
                brain.setArchitectureIn(None, option=optionIn, I=I, seed=k*100, **kwargsIn)
                brain.setWeightsIn(None, distribution='uniform', seed=k*100)
                
                # Readout initialization - multiple init
                brain.setArchitectureOut(None, distribution='uniform', seed=k*100, **kwargsOut)

                ##################################
                # Training
                ##################################
                print('------------------------')
                # Set input and target trains
                u_train = [inputs_train]
                y_train = [y_train]
                
                # TRAIN !
                brain.train(u_trains = u_train,
                            y_targets = y_train,
                            I = I,
                            duration = tTrain,
                            discardTime = tDiscard,
                            convergingTime = tConv,
                            option = learning,
                            optimizer = optimizer,
                            tag = tag,
                            **kwargs)
                print('Mean weight out :', np.mean(brain.Wout), np.std(brain.Wout))
                
                ##################################
                # Task on test set
                ##################################
                brain.reset()
                brain.activationFunction(activation)
                print('------------------------')
                print('Task : white-noise on test data')
                # test set fitting
                datas = rescale(whiteNoiseGauss(2*nbSample))
                y_test = datas[tDiscard+delay:nbSample+delay]
                u_test = datas[:tTrain]
                tTest = len(u_test)
                t_test  = np.arange(0, tTest)
                inputs = (u_test, t_test)
                y_pred, err = brain.read(inputs, duration=tTest, I=I,
                                discardTime=tDiscard, convergingTime=tConv,
                                y_target=y_test, reset=True, tag=tag, **kwargs)
                corrCoeff = my_corrcoef(y_pred, y_test)
                
                ##################################
                # Performance and storing
                ##################################
                # Error for each tau of MG
                print('Total error on that task :')
                print('Corr_coeff =', corrCoeff)
                corrTrials.append(corrCoeff)
                print('Done')
                print('------------------------')
            print('==================================')
            corr_delay[delay][sigma].append(corrTrials)
            infoReservoir[delay][sigma].append(brain.metadata)
            print(brain.metadata)

        # Save metadata
        tag_ = tag + f'_seed{seed}'
        brain.saveData(path=pathBrain, tag=tag_)
        brain.saveMetadata(path=pathBrain, tag=tag_)
        brain.reset(deep=True)
        
        # Save result in a dictionary]
        dicResult['corr'].update(corr_delay)
        dicResult['infoReservoir'].update(infoReservoir)
        np.save(name, dicResult)
    


  0%|                                                                  | 0/2 [00:00<?, ?it/s]

Reservoir #0, sigma=0.01
addReadoutLayer identity None
addReadoutLayer identity None
SCRIPT: C:\Users\Manu\Documents\GitHub\binary_model_design
delay=-10, sigma=0.01
0 already runned circuits
-----------------------------
Reservoir #0, trial #0
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -10
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
------------------------
---- Train readout ----
--> pytorch regression
---- Read - train ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> I

Training converged after 499 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : 0.00012336171 0.008249062
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.026246930763509008
Total error on that task :
Corr_coeff = 8.160443404245849e-16
Done
------------------------
Reservoir #0, trial #1
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -2
Format dimension for: instantiateArchitect

C:\Users\Manu\Documents\GitHub\binary_model_design\model\utils.py:1136: RuntimeWarning: invalid value encountered in double_scalars
  return (( x - mean_x ) * ( y - mean_y )).sum() / n / ( std_x * std_y )


Training converged after 599 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : 7.319692e-05 0.008159994
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.01710122951579244
Total error on that task :
Corr_coeff = nan
Done
------------------------
Reservoir #0, trial #3
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -2
Format dimension for: instantiateArchitecture
Dimension after 

...

Training converged after 599 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : 0.00019449324 0.008134364
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.017101228872332386
Total error on that task :
Corr_coeff = 5.217836457889463e-16
Done
------------------------
Reservoir #12, trial #3
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -2
Format dimension for: instantiateArchitec

Training converged after 299 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : -2.23786e-05 0.0080678705
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.01731854740293742
Total error on that task :
Corr_coeff = -2.3656732608039427e-16
Done
------------------------
Reservoir #13, trial #3
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -10
Format dimension for: instantiateArchit

Training converged after 599 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : -7.366704e-05 0.008151183
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.017101229837522634
Total error on that task :
Corr_coeff = 5.23669413207731e-16
Done
------------------------
Reservoir #13, trial #3
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -2
Format dimension for: instantiateArchitect

Training converged after 499 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : 5.0088525e-05 0.008159813
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.016929006247031732
Total error on that task :
Corr_coeff = -2.3656732608039427e-16
Done
------------------------
Reservoir #14, trial #3
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -10
Format dimension for: instantiateArchi

Training converged after 699 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
---- Readout trained ----
Mean weight out : 0.0001360263 0.008191875
------------------------
Task : white-noise on test data
---- Read - output ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 500
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 500
y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.01710122903319736
Total error on that task :
Corr_coeff = -5.217836457889463e-16
Done
------------------------
Reservoir #14, trial #3
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
White-noise, delay = -2
Format dimension for: instantiateArchitect

In [ ]:
# Format all results in a dataframe
import pandas as pd
from binary_model.utils import save_df, add_balance, load_df


def make_csv_corr_vs_K(N, mu, Ks, nbtrial, nbReservoir, 
                       delays, output, optimizer, 
                       activation, sim='performance_task',
                       experiment='prediction_mackey-glass'):
    
    # Save all results
    result_dir = dataPathPerf(sim, experiment)
    commonFolder = os.path.join(result_dir, 'allResults/')
    if not os.path.isdir(commonFolder):
        os.makedirs(commonFolder)
    
    for K in Ks:
        df_name = f'{experiment}_allReservoirs_N{N}_K{K}_W{mu}_T{nbtrial}'
        df_path = commonFolder + df_name

        path = dataPathPerf(sim, experiment, optimizer=optimizer,
                            activation=activation, nbtrial=nbtrial)

        tag = f"{experiment}_{optimizer}_{activation}_output-{output}"
        name = f'/dicError_{tag}_nbReservoir{nbReservoir}_N{N}_K{K}_W{mu}_T{nbtrial}.npy'

        print(path+name)
        dicResult = np.load(path+name, allow_pickle='TRUE').item()

        infoReservoirs = dicResult['infoReservoir']
        corrReservoirs = dicResult['corr']
        for delay in delays:
            corrReservoir_sigma = corrReservoirs[delay]
            infoReservoir_sigma = infoReservoirs[delay]

            sigmas = []
            seeds = []
            corr_reservoirs = []
            for n, (sigma, corr_list) in enumerate(corrReservoir_sigma.items()):
                infoReservoir_list = infoReservoir_sigma[sigma]

                for i, corr in enumerate(corr_list[0:nbReservoir]):

                    # Load data
                    infoReservoir = infoReservoir_list[i]
                    sigma = infoReservoir['stdWeights']
                    # Update results
                    sigmas.append(sigma)
                    seeds.append(seed)
                    corr = np.array(corr)
                    corr = corr[~np.isnan(corr)]
                    corr_reservoirs.append(np.mean(corr))

            # Create dictionary
            dic_reservoir = {'stdWeights':sigmas,
                           'seed':seeds,
                           f'<C>, delay={delay}':corr_reservoirs,
                           }
            # Save csv
            df = save_df(df_path, dic_reservoir)
            add_balance(df, df_name, commonFolder, N=N, K=K, W=W, T=T, d=d)
    
make_csv_corr_vs_K(N, mu, Ks, nb_trial, nb_reservoir, 
                   delays, output_option, optimizer, 
                   activation, sim=sim,
                   experiment=experiment)

In [ ]:
name=f'{experiment}_allReservoirs'
result_dir = dataPathPerf(sim, experiment)
path = os.path.join(result_dir, 'allResults/')
df = load_df(name, path, N=N, K=K, W=W, T=T)
df.head()